In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

from trading_system import TradingSystem

In [2]:
hub1_name = "ttf"
hub2_name = "nbp"
horizon = 250
window_size = 5
model = "tvecm_t1"

ts = TradingSystem(hub1_name, hub2_name, model, horizon, window_size)


In [3]:
ts.run_trading_system(rolling_window=60, threshold=0.5)

Profit:  14.78096168363645


In [4]:
profit = ts.get_profit()
mean_returns, mean_returns_with_tc, ci_returns, ci_returns_with_tc = ts.get_returns_stats()
print(f"Profit: {profit:.2f}")
print(f"Mean returns: {mean_returns:.2f}%")
print(f"Mean returns with transaction costs: {mean_returns_with_tc:.2f}%")
print(f"Confidence interval (returns): {ci_returns[0]:.2f}% - {ci_returns[1]:.2f}%")
print(f"Confidence interval (returns with transaction costs): {ci_returns_with_tc[0]:.2f}% - {ci_returns_with_tc[1]:.2f}%")

Profit: 14.78
Mean returns: 0.17%
Mean returns with transaction costs: 0.17%
Confidence interval (returns): 0.08% - 0.26%
Confidence interval (returns with transaction costs): 0.08% - 0.26%


In [5]:
trade_rates = ts.get_trade_rates()
print(f"Win rate for returns: {trade_rates['win_rate_returns']:.2%}")
print(f"No trade rate for returns: {trade_rates['no_trade_rate_returns']:.2%}")
print(f"Loss rate for returns: {trade_rates['loss_rate_returns']:.2%}")
print(f"Win rate for returns with transaction costs: {trade_rates['win_rate_returns_with_tc']:.2%}")
print(f"No trade rate for returns with transaction costs: {trade_rates['no_trade_rate_returns_with_tc']:.2%}")
print(f"Loss rate for returns with transaction costs: {trade_rates['loss_rate_returns_with_tc']:.2%}")


Win rate for returns: 17.20%
No trade rate for returns: 75.20%
Loss rate for returns: 7.60%
Win rate for returns with transaction costs: 17.20%
No trade rate for returns with transaction costs: 75.20%
Loss rate for returns with transaction costs: 7.60%


In [20]:
fig = go.Figure()

# Add line plot for pl
fig.add_trace(go.Scatter(
    x=list(range(horizon)),
    y=ts.get_pl(),
    mode='lines',
    name='Profit/Loss'
))

# Add scatter plot for trades
fig.add_trace(go.Scatter(
    x=list(range(horizon)),
    y=ts.get_trades(),
    mode='markers',
    name='Trades',
    yaxis='y2'
))

# Create axis objects
fig.update_layout(
    title='Profit/Loss and Trades Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Profit/Loss'),
    yaxis2=dict(title='Trades', overlaying='y', side='right')
)

fig.show()

In [8]:




if "arima" in model:
    hub1_data = f"{hub1_name}_h{horizon}_w{window_size}"
    predictions = pd.read_csv(f"../predictions/{hub1_data}_{model}_predictions.csv")
    predictions = predictions.set_index("Date")
    hub1_predictions = predictions[[hub1_name]]

    hub2_data = f"{hub2_name}_h{horizon}_w{window_size}"
    predictions = pd.read_csv(f"../predictions/{hub2_data}_{model}_predictions.csv")
    predictions = predictions.set_index("Date")
    hub2_predictions = predictions[[hub2_name]]
else:
    data = f"{hub1_name}_{hub2_name}_h{horizon}_w{window_size}"
    predictions = pd.read_csv(f"../predictions/{data}_{model}_predictions.csv")
    predictions = predictions.set_index("Date")
    hub1_predictions = predictions[[hub1_name]]
    hub2_predictions = predictions[[hub2_name]]
    




hub1_historical_data = pd.read_csv(f"../data/interpolated/{hub1_name}_close_interpolated.csv")
hub1_historical_data = hub1_historical_data.set_index("Date")

hub2_historical_data = pd.read_csv(f"../data/interpolated/{hub2_name}_close_interpolated.csv")
hub2_historical_data = hub2_historical_data.set_index("Date")

hub1_historical_data.rename(columns={"CLOSE": hub1_name}, inplace=True)
hub2_historical_data.rename(columns={"CLOSE": hub2_name}, inplace=True)

hub1_last_available_data = hub1_historical_data[-(horizon + window_size):-window_size].values
hub2_last_available_data = hub2_historical_data[-(horizon + window_size):-window_size].values

hub1_actual_data = hub1_historical_data[-horizon:].values
hub2_actual_data = hub2_historical_data[-horizon:].values

In [9]:

hub1_last_available_data = hub1_historical_data[-(horizon + window_size):-window_size][[hub1_name]]
hub1_actuals = hub1_historical_data[-horizon:][[hub1_name]]

hub1_predicted_returns = np.log(hub1_predictions.values / hub1_last_available_data)*100


hub2_last_available_data = hub2_historical_data[-(horizon + window_size):-window_size][[hub2_name]]
hub2_actuals = hub2_historical_data[-horizon:][[hub2_name]]

hub2_predicted_returns = np.log(hub2_predictions.values / hub2_last_available_data)*100


In [10]:
fig = px.line(
    x=hub1_predicted_returns.index,
    y=[hub1_predicted_returns[hub1_name], hub2_predicted_returns[hub2_name]],
    labels={'x': 'Date', 'value': 'Predicted Returns', 'variable': 'Hub'},
    title='Predicted Returns for Hub1 and Hub2'
)

# Update the names for each line in the legend
fig.data[0].name = hub1_name  # Assign custom name for hub1 in the legend
fig.data[1].name = hub2_name  # Assign custom name for hub2 in the legend

fig.update_layout(
    legend_title_text='Hub',
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)

fig.show()


In [11]:
fig = px.line(
    x=hub1_actuals.index,
    y=[hub1_actuals[hub1_name], hub2_actuals[hub2_name]],
    labels={'x': 'Date', 'value': 'Predicted Returns', 'variable': 'Hub'},
    title='Actual Prices for Hub1 and Hub2'
)

# Update the names for each line in the legend
fig.data[0].name = hub1_name  # Assign custom name for hub1 in the legend
fig.data[1].name = hub2_name  # Assign custom name for hub2 in the legend

fig.update_layout(
    legend_title_text='Hub',
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    )
)

fig.show()


In [12]:
forecast_horizon = len(hub1_predicted_returns)

In [13]:
hub1_historical_returns = np.log(hub1_historical_data[window_size:][[hub1_name]].values / hub1_historical_data[:-window_size][[hub1_name]])*100
hub2_historical_returns = np.log(hub2_historical_data[window_size:][[hub2_name]].values / hub2_historical_data[:-window_size][[hub2_name]])*100
returns_difference = pd.DataFrame(hub1_historical_returns[hub1_name] - hub2_historical_returns[hub2_name], columns=['difference'])

In [14]:
def trading_system(forecast_horizon, 
                   hub1_predicted_returns, hub2_predicted_returns, 
                   hub1_last_available_data, hub2_last_available_data, 
                   hub1_actuals, hub2_actuals,
                   historical_returns_difference,
                   rolling_window=14,
                   threshold=2):
    

    transaction_costs = 0.005

    trades = [0] * forecast_horizon
    pl = [0] * forecast_horizon
    returns = [0] * forecast_horizon
    returns_with_transaction_costs = [0] * forecast_horizon

    profit = 0

    for i in range(forecast_horizon):
        hub1_predicted_return = hub1_predicted_returns.values[i].item()
        hub2_predicted_return = hub2_predicted_returns.values[i].item()

        returns_difference = hub1_predicted_return - hub2_predicted_return
        rolling_data = historical_returns_difference[["difference"]][-forecast_horizon - window_size - rolling_window + i :-forecast_horizon - window_size + i]
        returns_difference_std = rolling_data.std().item()
        returns_difference_mean = rolling_data.mean().item()

        if returns_difference >  threshold * returns_difference_std:

            long_position_return = np.log(hub1_actuals.values[i].item() / hub1_last_available_data.values[i].item())
            short_position_return = - np.log(hub2_actuals.values[i].item() / hub2_last_available_data.values[i].item())

            long_position_return_with_transaction_costs = np.log((hub1_actuals.values[i].item() - 2*transaction_costs)/ hub1_last_available_data.values[i].item())
            short_position_return_with_transaction_costs = - np.log((hub2_actuals.values[i].item() - 2*transaction_costs) / hub2_last_available_data.values[i].item())


            net_profit_long = hub1_actuals.values[i].item() - hub1_last_available_data.values[i].item()
            net_profit_short = hub2_last_available_data.values[i].item() - hub2_actuals.values[i].item()

            monetary_neutral_adjustment = hub1_last_available_data.values[i].item() / hub2_last_available_data.values[i].item()

            profit += net_profit_long + monetary_neutral_adjustment * net_profit_short - transaction_costs*4


            trades[i] = 1
            returns[i] = long_position_return + short_position_return
            returns_with_transaction_costs[i] = long_position_return_with_transaction_costs + short_position_return_with_transaction_costs

        elif returns_difference <  -  threshold * returns_difference_std:

            long_position_return = np.log(hub2_actuals.values[i].item() / hub2_last_available_data.values[i].item())
            short_position_return = - np.log(hub1_actuals.values[i].item() / hub1_last_available_data.values[i].item())

            long_position_return_with_transaction_costs = np.log((hub2_actuals.values[i].item() - 2*transaction_costs)/ hub2_last_available_data.values[i].item())
            short_position_return_with_transaction_costs = - np.log((hub1_actuals.values[i].item() - 2*transaction_costs) / hub1_last_available_data.values[i].item())

            net_profit_long = hub2_actuals.values[i].item() - hub2_last_available_data.values[i].item()
            net_profit_short = hub1_last_available_data.values[i].item() - hub1_actuals.values[i].item()

            monetary_neutral_adjustment = hub2_last_available_data.values[i].item() / hub1_last_available_data.values[i].item()

            profit += net_profit_long + monetary_neutral_adjustment * net_profit_short - transaction_costs*4

            trades[i] = 1
            returns[i] = long_position_return + short_position_return
            returns_with_transaction_costs[i] = long_position_return_with_transaction_costs + short_position_return_with_transaction_costs

        else:
            pass

        pl[i] = profit

    return pl, profit, trades, returns, returns_with_transaction_costs

In [15]:
pl, profit, trades, returns, returns_with_transaction_costs = trading_system(
    forecast_horizon, 
    hub1_predicted_returns, hub2_predicted_returns, 
    hub1_last_available_data, hub2_last_available_data, 
    hub1_actuals, hub2_actuals,
    returns_difference,
    rolling_window=5,
    threshold=0)

print(f"Profit: {profit}")
print(np.mean(returns)*100)
print(np.mean(returns_with_transaction_costs)*100)
standard_error_returns = np.std(returns) / np.sqrt(len(returns))
print((np.mean(returns) - 1.96 * standard_error_returns)*100, (np.mean(returns) + 1.96 * standard_error_returns)*100)

standard_error_returns_with_transaction_costs = np.std(returns_with_transaction_costs) / np.sqrt(len(returns_with_transaction_costs))
print((np.mean(returns) - 1.96 * standard_error_returns_with_transaction_costs)*100, (np.mean(returns) + 1.96 * standard_error_returns_with_transaction_costs)*100)


Profit: 1.44637720772119
0.06990022028959927
0.06996712164934644
-0.10538862507723736 0.24518906565643592
-0.10541632738271718 0.24521676796191572


In [576]:
# Calculate win and loss rates for returns
win_rate_returns = sum(1 for r in returns if r > 0) / len(returns)
no_trade_rate_returns = sum(1 for r in returns if r == 0) / len(returns)
loss_rate_returns = sum(1 for r in returns if r < 0) / len(returns)

# Calculate win and loss rates for returns with transaction costs
win_rate_returns_with_tc = sum(1 for r in returns_with_transaction_costs if r > 0) / len(returns_with_transaction_costs)
no_trade_rate_returns_with_tc = sum(1 for r in returns_with_transaction_costs if r == 0) / len(returns_with_transaction_costs)
loss_rate_returns_with_tc = sum(1 for r in returns_with_transaction_costs if r < 0) / len(returns_with_transaction_costs)

print(f"Win rate for returns: {win_rate_returns:.2%}")
print(f"No trade rate for returns: {no_trade_rate_returns:.2%}")
print(f"Loss rate for returns: {loss_rate_returns:.2%}")
print(f"Win rate for returns with transaction costs: {win_rate_returns_with_tc:.2%}")
print(f"No trade rate for returns with transaction costs: {no_trade_rate_returns_with_tc:.2%}")
print(f"Loss rate for returns with transaction costs: {loss_rate_returns_with_tc:.2%}")

Win rate for returns: 53.20%
No trade rate for returns: 0.00%
Loss rate for returns: 46.80%
Win rate for returns with transaction costs: 53.20%
No trade rate for returns with transaction costs: 0.00%
Loss rate for returns with transaction costs: 46.80%


In [563]:
fig = go.Figure()

# Add line plot for pl
fig.add_trace(go.Scatter(
    x=list(range(forecast_horizon)),
    y=pl,
    mode='lines',
    name='Profit/Loss'
))

# Add scatter plot for trades
fig.add_trace(go.Scatter(
    x=list(range(forecast_horizon)),
    y=trades,
    mode='markers',
    name='Trades',
    yaxis='y2'
))

# Create axis objects
fig.update_layout(
    title='Profit/Loss and Trades Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Profit/Loss'),
    yaxis2=dict(title='Trades', overlaying='y', side='right')
)

fig.show()

In [569]:
np.random.seed(42)


def naive_trading_system(forecast_horizon, 
                   hub1_predicted_returns, hub2_predicted_returns, 
                   hub1_last_available_data, hub2_last_available_data, 
                   hub1_actuals, hub2_actuals,
                   historical_returns_difference,
                   rolling_window=14,
                   threshold=2):
    

    transaction_costs = 0.005

    trades = [0] * forecast_horizon
    pl = [0] * forecast_horizon
    returns = [0] * forecast_horizon
    returns_with_transaction_costs = [0] * forecast_horizon

    profit = 0

    for i in range(forecast_horizon):
        hub1_predicted_return = hub1_predicted_returns.values[i].item()
        hub2_predicted_return = hub2_predicted_returns.values[i].item()

        returns_difference = hub1_predicted_return - hub2_predicted_return
        rolling_data = historical_returns_difference[["difference"]][-forecast_horizon - window_size - rolling_window + i :-forecast_horizon - window_size + i]
        returns_difference_std = rolling_data.std().item()
        returns_difference_mean = rolling_data.mean().item()

        if np.random.rand() > 0.5:

            long_position_return = np.log(hub1_actuals.values[i].item() / hub1_last_available_data.values[i].item())
            short_position_return = - np.log(hub2_actuals.values[i].item() / hub2_last_available_data.values[i].item())

            long_position_return_with_transaction_costs = np.log((hub1_actuals.values[i].item() - 2*transaction_costs)/ hub1_last_available_data.values[i].item())
            short_position_return_with_transaction_costs = - np.log((hub2_actuals.values[i].item() - 2*transaction_costs) / hub2_last_available_data.values[i].item())


            net_profit_long = hub1_actuals.values[i].item() - hub1_last_available_data.values[i].item()
            net_profit_short = hub2_last_available_data.values[i].item() - hub2_actuals.values[i].item()

            monetary_neutral_adjustment = hub1_last_available_data.values[i].item() / hub2_last_available_data.values[i].item()

            profit += net_profit_long + monetary_neutral_adjustment * net_profit_short - transaction_costs*4


            trades[i] = 1
            returns[i] = long_position_return + short_position_return
            returns_with_transaction_costs[i] = long_position_return_with_transaction_costs + short_position_return_with_transaction_costs

        else:

            long_position_return = np.log(hub2_actuals.values[i].item() / hub2_last_available_data.values[i].item())
            short_position_return = - np.log(hub1_actuals.values[i].item() / hub1_last_available_data.values[i].item())

            long_position_return_with_transaction_costs = np.log((hub2_actuals.values[i].item() - 2*transaction_costs)/ hub2_last_available_data.values[i].item())
            short_position_return_with_transaction_costs = - np.log((hub1_actuals.values[i].item() - 2*transaction_costs) / hub1_last_available_data.values[i].item())

            net_profit_long = hub2_actuals.values[i].item() - hub2_last_available_data.values[i].item()
            net_profit_short = hub1_last_available_data.values[i].item() - hub1_actuals.values[i].item()

            monetary_neutral_adjustment = hub2_last_available_data.values[i].item() / hub1_last_available_data.values[i].item()

            profit += net_profit_long + monetary_neutral_adjustment * net_profit_short - transaction_costs*4

            trades[i] = 1
            returns[i] = long_position_return + short_position_return
            returns_with_transaction_costs[i] = long_position_return_with_transaction_costs + short_position_return_with_transaction_costs

        pl[i] = profit

    return pl, profit, trades, returns, returns_with_transaction_costs

In [577]:
pl, profit, trades, returns, returns_with_transaction_costs = naive_trading_system(
    forecast_horizon, 
    hub1_predicted_returns, hub2_predicted_returns, 
    hub1_last_available_data, hub2_last_available_data, 
    hub1_actuals, hub2_actuals,
    returns_difference,
    rolling_window=5,
    threshold=0)

print(f"Profit: {profit}")
print(np.mean(returns)*100)
print(np.mean(returns_with_transaction_costs)*100)
standard_error_returns = np.std(returns) / np.sqrt(len(returns))
print((np.mean(returns) - 1.96 * standard_error_returns)*100, (np.mean(returns) + 1.96 * standard_error_returns)*100)

standard_error_returns_with_transaction_costs = np.std(returns_with_transaction_costs) / np.sqrt(len(returns_with_transaction_costs))
print((np.mean(returns) - 1.96 * standard_error_returns_with_transaction_costs)*100, (np.mean(returns) + 1.96 * standard_error_returns_with_transaction_costs)*100)


Profit: 0.3458023509201793
0.0394200916522963
0.03947463188320576
-0.1360147442498186 0.21485492755441118
-0.13604264482844786 0.21488282813304044
